# Path of Exile Trade

Path of exile 2 (POE2) is a multi-player game with a open marketplace where players can trade equipment with each other.

Just like the real world, these equipment are traded in different currencies, but rather than USD, SGD, EUR, etc, the game instead has currencies that look like the following:
1. DIVINE ORBS
2. EXLATED ORBS
3. CHAOS ORBS
4. etc.

![different_currencies](images/different_currencies.png)

Next, we should observe that for a given class of equipment, they come with different attributes, for example, for 2 wands, they can differ in the following:
1. The first wand requires a level of 46 to wield, whilst the second wand requires a level of 72 to wield (REQUIREMENTS)
2. The first wand requires an intelligence of 106 to wield, whilst the second wand requires an intelligence of 164 to wield (REQUIREMENTS)
3. The first wand grants 25% SPELL DAMAGE, whilst the second wand grants GAIN 19% OF DAMAGE AS EXTRA FIRE DAMAGE (ATTRIBUTES)
4. etc...

![different_attributes](images/different_attributes.png)

#### The purpose of this take-home assignment is to:
1. Scrape the website for current items -> prices
2. The BASE currency that you should work with is EXALTED ORBS, but as a special bonus, you should make the algorithm flexible to the BASE CURRENCY of choice.
3. The reason why you need to define a base currency is because there are many different currencies you can calculate prices in.
4. Come up with an algorithm that can calculate the fair price of an item in the BASE CURRENCY, having some assumptions about outliers (hint) and averages (hint)
5. For every item, identify their fair price in the BASE CURRENCY
6. Identify every item that is being sold for below their fair price in the BASE CURRENCY by a certain percentage margin (e.g. 25%)
7. This requires you to convert the QUOTE CURRENCY of the item into the BASE CURRENCY, which requires you to estimate QUOTE CURRENCY: BASE CURRENCY before calculation (hint)
8. This algorithm should be able to ran live and reflect the latest items being misvalued when demonstrated
9. For the purposes of this take-home assignment, we can focus only on ITEM CATEGORY: WAND, but the algorithm should generalize to any other item categories

#### Notes:
1. You should definitely create an account and take awhile to familiarize yourself with the market @ `https://www.pathofexile.com/trade2/search/poe2/Rise%20of%20the%20Abyssal`
2. You should only work with `Standard` as opposed to the other `Leagues`
3. `Corrupted` is an important attribute to include in your pricing, there are other important hidden attributes as well that you should take note of

#### Useful Hints:
1. You will need to scrape mapping of currencies to other currencies to convert the quote currencies to the base currencies in real time
2. You will need to figure out the fair value of currencies (they are greatly affected by outliers, be careful)
3. Outliers play a big role in this assignment, figuring out how to solve for them is a big part of it
4. Both REQUIREMENTS and ATTRIBUTES have a part to play in price, you will need to include REQUIREMENTS in your pricing algorithm
5. In order to work on this project, you will need to make some (strong) assumptions, list down all the assumptions that you think are important
6. We will test this out-of-sample by looking at your ability to find underpriced items in real-time

# Getting Current Items

To get current items -> prices, you need to scrape `https://www.pathofexile.com/trade2/search/poe2/Rise%20of%20the%20Abyssal`

You pass in a payload and get back a list of IDs that meet your filters

In [6]:
import requests

payload = {
    "query":{
        "status":{"option":"online"},
        "stats":[{"type":"and","filters":[],"disabled":False}],
        "filters":{"type_filters":{"filters":{"category":{"option":"weapon.wand"}},"disabled":False}}},
    "sort":{"price":"asc"}
}

session = requests.Session()

headers = {
    # Normal headers (mapped from your DevTools capture)
    "Accept": "*/*",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept-Language": "en-US,en;q=0.9",
    "Content-Type": "application/json",
    "Origin": "https://www.pathofexile.com",
    "Priority": "u=1, i",
    "Referer": "https://www.pathofexile.com/trade2/search/poe2/Rise%20of%20the%20Abyssal",
    "Sec-CH-UA": "\"Not;A=Brand\";v=\"99\", \"Brave\";v=\"139\", \"Chromium\";v=\"139\"",
    "Sec-CH-UA-Arch": "\"x86\"",
    "Sec-CH-UA-Bitness": "\"64\"",
    "Sec-CH-UA-Full-Version-List": "\"Not;A=Brand\";v=\"99.0.0.0\", \"Brave\";v=\"139.0.0.0\", \"Chromium\";v=\"139.0.0.0\"",
    "Sec-CH-UA-Mobile": "?0",
    "Sec-CH-UA-Model": "",
    "Sec-CH-UA-Platform": "\"Windows\"",
    "Sec-CH-UA-Platform-Version": "\"19.0.0\"",
    "Sec-Fetch-Dest": "empty",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Site": "same-origin",
    "Sec-GPC": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/139.0.0.0 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest",
}

url = "https://www.pathofexile.com/api/trade2/search/poe2/Rise%20of%20the%20Abyssal"

scrapped_wands = requests.post(
    url,
    json=payload,
    headers=headers,
    timeout=30,
)

In [7]:
scrapped_wands_json = scrapped_wands.json()

# Now we need to fetch for a single wand

In [8]:
scrapped_wands_json['id']

'yqmP2nefR'

In [9]:
scrapped_wands_json['result'][1]

'72f5b85a37251884378b7df050250486e9a216a44eac516d7918b95c0271d484'

In [10]:
single_wand_url = "https://www.pathofexile.com/api/trade2/fetch/" + scrapped_wands_json['result'][1]

single_wand_payload = {
    "query": scrapped_wands_json['id'],
    "realm": "poe2"
}

single_wand = requests.post(
    single_wand_url,
    json=single_wand_payload,
    headers=headers,
    timeout=30,
)

In [11]:
single_wand.json()

{'result': [{'id': '72f5b85a37251884378b7df050250486e9a216a44eac516d7918b95c0271d484',
   'listing': {'method': 'psapi',
    'indexed': '2025-08-29T22:00:33Z',
    'stash': {'name': '6', 'x': 0, 'y': 8},
    'price': {'type': '~price', 'amount': 1, 'currency': 'alch'},
    'account': {'name': 'WZJ#5044',
     'online': {'league': 'Rise of the Abyssal'},
     'lastCharacterName': '蟹子莱莱',
     'language': 'zh_TW',
     'realm': 'poe2'},
    'whisper': '@蟹子莱莱 你好，我想購買 凋零法杖 標價 1 alch 在 Rise of the Abyssal (倉庫頁 "6"; 位置: 左 1, 上 9)'},
   'item': {'realm': 'poe2',
    'verified': True,
    'w': 1,
    'h': 3,
    'icon': 'https://web.poecdn.com/gen/image/WzI1LDE0LHsiZiI6IjJESXRlbXMvV2VhcG9ucy9PbmVIYW5kV2VhcG9ucy9XYW5kcy9CYXNldHlwZXMvV2FuZDAxIiwidyI6MSwiaCI6Mywic2NhbGUiOjEsInJlYWxtIjoicG9lMiJ9XQ/8b127e7050/Wand01.png',
    'league': 'Rise of the Abyssal',
    'id': '72f5b85a37251884378b7df050250486e9a216a44eac516d7918b95c0271d484',
    'name': '',
    'typeLine': 'Withered Wand',
    'baseType':

# Why This Take Home Project

Firstly, this is quite a difficult project that requires cross-domain skills, which is something we work with every single day.
It requires you to have:
1. Sufficient dev skills to navigate working with requests, I/O, concurrency etc. and put together a decent structure to bring this project to life.
2. Sufficent quant researcher skills to approach solving pricing AND
3. Sufficient data scientist skills to arrange messy data into structured clean data and to build the necessary models to solve this (hint) AND
4. Being extremely autonomous and resourceful - much of this project will require you to FIGURE THINGS OUT and GET THE INFORMATION YOU NEED, all traits of people we love working with.
5. Be intelligent enough to abstract away the distractions. It does not matter that you don't know how to play the game.

Secondly, we want hard projects that ChatGPT is not trained to solve so as to encourage original thinking. You are free to use any tools you need to use to solve this. But you will still need to be the one coming up with the assumptions and algorithms, because ChatGPT would not have seen this solution.

# What Do We Expect?

1. Deliver a well-defined, well-written Class with a method that can take in a base_currency, a percentage discount, and a `filter`, and returns a list of all items of a given `filter` that is sold for less than their percentage discount on the fair price.
2. To do that, you first need to have a model that can determine the fair price of all items.